In [46]:
import os
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from pycocotools.coco import COCO

from PIL import Image

In [3]:
# Path to the image directory and annotation file
dataset_path = '../data'
anns_file_path = dataset_path + '/' + 'annotations.json'

# Load the annotations
coco = COCO(anns_file_path)

# Create a dictionary of the label names and their corresponding IDs
labels = {}
for i, cat in enumerate(coco.loadCats(coco.getCatIds())):
    labels[cat['name']] = i
    
#labels

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [ ]:
import os

# specify the directory path
dir_path = '/path/to/directory'

# list all the files in the directory
file_list = os.listdir(dir_path)

# select the second file name
second_file = file_list[1]

print(second_file)


In [114]:
IMAGE_DIR = '/Users/mjs/Desktop/Dev/TACO/data/all_images'

# Set the batch size and image size
BATCH_SIZE = 16
IMG_SIZE = (224, 224)

# Load the image file names and labels from a CSV file
df = pd.read_csv('../data/filtered_labels.csv')



In [120]:
datagen_train=tf.keras.preprocessing.image.ImageDataGenerator(
                      rescale=1./255,
                      rotation_range=15,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.005,
                      zoom_range=[0.9, 1.4],
                      horizontal_flip=True,
                      vertical_flip=False,
                      brightness_range=(.8,1.2),
                      fill_mode='nearest',
                      )

train_generator=datagen_train.flow_from_dataframe(
    dataframe=df,
    directory=IMAGE_DIR,
    shuffle=True,
    x_col='file_name',
    y_col=['Aluminium_blister_pack', 'Carded_blister_pack', 'Other_plastic_bottle', 'Clear_plastic_bottle'],
  # save_to_dir=savepath + '/aug_images', 
    #classes=train_classes,
    class_mode='multi_output',
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=batch_size)

train_generator

Found 833 validated image filenames.


In [122]:
df = pd.read_csv('../data/filtered_labels.csv')
df.head()
type(False)
type(df['Aluminium_foil'][0])
df = df.astype(str)
type(df['Aluminium_foil'][0])

labels = list(df.columns)[1:]





In [128]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split

# Define hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 6
EPOCHS = 10
LEARNING_RATE = 0.0001

# Load data from CSV file
df = pd.read_csv('../data/filtered_labels.csv')

labels = list(df.columns)[1:]

# Split data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2)

# Define data generators for train and validation sets
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.005,
    zoom_range=[0.9, 1.4],
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=(.8,1.2),
    fill_mode='nearest',
)

datagen_val = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    directory='/Users/mjs/Desktop/Dev/TACO/data/all_images',
    x_col='file_name',
    y_col=labels,
    class_mode='raw',
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_generator = datagen_val.flow_from_dataframe(
    dataframe=val_df,
    directory='/Users/mjs/Desktop/Dev/TACO/data/all_images',
    x_col='file_name',
    y_col=labels,
    class_mode='raw',
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Load the pre-trained EfficientNetB0 model and its weights
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze all the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a custom classification head
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = Dense(len(labels), activation='sigmoid')(x)

# Create the final model by combining the base model with the custom head
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
optimizer = Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# Train the model
# Train the model
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    #callbacks=[checkpoint]
                   )

# Convert the history object to a dictionary
history_dict = history.history
# Convert any numpy arrays to lists so they can be serialized by JSON
for key, val in history_dict.items():
    if isinstance(val, np.ndarray):
        history_dict[key] = val.tolist()
# Save the history dictionary to a JSON file
with open('history.json', 'w') as outfile:
    json.dump(history_dict, outfile)

Found 666 validated image filenames.
Found 167 validated image filenames.


/Users/mjs/Conda/anaconda3/envs/DLG/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
111/111 [==============================] - 138s 1s/step - loss: 0.2231 - accuracy: 0.0796 - val_loss: 0.1236 - val_accuracy: 0.0060
Epoch 2/10
111/111 [==============================] - 139s 1s/step - loss: 0.1225 - accuracy: 0.1036 - val_loss: 0.1216 - val_accuracy: 0.1377
Epoch 3/10
111/111 [==============================] - 137s 1s/step - loss: 0.1216 - accuracy: 0.0811 - val_loss: 0.1214 - val_accuracy: 0.0060
Epoch 4/10
111/111 [==============================] - 137s 1s/step - loss: 0.1214 - accuracy: 0.0961 - val_loss: 0.1215 - val_accuracy: 0.1377
Epoch 5/10
111/111 [==============================] - 141s 1s/step - loss: 0.1212 - accuracy: 0.0871 - val_loss: 0.1219 - val_accuracy: 0.1377
Epoch 6/10
111/111 [==============================] - 138s 1s/step - loss: 0.1216 - accuracy: 0.1021 - val_loss: 0.1222 - val_accuracy: 0.1377
Epoch 7/10
111/111 [==============================] - 143s 1s/step - loss: 0.1212 - accuracy: 0.1036 - val_loss: 0.1229 - val_accuracy: 0.0060

In [129]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_generator)
print(f'Validation loss: {loss:.3f}')
print(f'Validation accuracy: {accuracy:.3f}')

28/28 [==============================] - 25s 894ms/step - loss: 0.1221 - accuracy: 0.0060
Validation loss: 0.122
Validation accuracy: 0.006
